In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plot
import matplotlib
import chart_studio.plotly as py
import plotly.graph_objs as go 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import pycountry
import plotly.graph_objects as go

In [ ]:
def getDataFrameFromTable(tableName):
    df = pd.DataFrame()
    connection=None
    try:
        # Create an engine instance
        engine = postgresEngine
        # Connect to PostgreSQL server
        connection = engine.connect()
        df= pd.read_sql("select * from \""+tableName+"\" ", connection);
    except Exception as e:
        print('Exception in createDataFramefromTbl----->\n',e)
    finally:
        if connection is not None:
            connection.close()
    return df

In [ ]:
resultantDf = getDataFrameFromTable(resultantTableName)

In [ ]:
resultantDf

In [ ]:
groupByCountryDf = resultantDf.loc[:,("population", "continent_name",'country_name','Air_Quality_Index','Carbon_Monoxide','Nitrogen_dioxide','Ozone','Fine_Particles_matter','Coarse_particulate_Matter','temperature','humidity', 'pressure')]
groupByCountryDf=groupByCountryDf.groupby(['country_name']).mean().reset_index()
groupByCountryDf

In [ ]:
groupByContinentDf = resultantDf.loc[:,("population", "continent_name",'country_name','Air_Quality_Index','Carbon_Monoxide','Nitrogen_dioxide','Ozone','Fine_Particles_matter','Coarse_particulate_Matter','temperature','humidity', 'pressure', 'wind_speed', 'wind_deg')]
groupByContinentDf=groupByContinentDf.groupby(['continent_name']).mean().reset_index()
groupByContinentDf

In [ ]:
data = dict(
        type = 'choropleth',
        colorscale = 'aggrnyl',
        locations = resultantDf['country_code'],
        z = resultantDf['population'],
        text = resultantDf['country_name'],
        colorbar = {'title' : 'GDP Billions US'},
      ) 

In [ ]:
def findCountry (country_name):
    try:
        return pycountry.countries.get(name=country_name).alpha_3
    except:
        return ("not founded")


In [ ]:
def parseCountryCodeInCountryDf():
    resultantDf['country_alpha_3'] = resultantDf.apply(lambda row: findCountry(row.country_name) , axis = 1);
parseCountryCodeInCountryDf()

## Which country in Europe has the highest density of population?

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations = resultantDf['country_alpha_3'],
    z = resultantDf['population'],
    text = resultantDf['country_name'],
    colorscale = 'Blues',
    autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '',
    colorbar_title = 'In<br>Millions',
    zmin=5,
    zmax=1000000
))

fig.update_layout(
    title_text='Population Distribution',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.55,
        y=0.1,
        xref='paper',
        yref='paper',
        text='Population density across the world',
        showarrow = False
    )]
)

fig.show()

## Will decreased levels of Ozone lead to increase in temperature? 

In [ ]:
graphPlot = sns.jointplot(x='Ozone',y='temperature',data=resultantDf,kind='scatter',color='#32c8cd')
graphPlot.fig.suptitle("Ozone vs Temperature Plot")
graphPlot.fig.tight_layout()
graphPlot.fig.subplots_adjust(top=0.90)
graphPlot.fig.set_size_inches(10,10)


## Which countries have very low ozone levels?

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations = resultantDf['country_alpha_3'],
    z = resultantDf['Ozone'],
    text = resultantDf['country_name'],
    colorscale = 'magma',
    autocolorscale=False,
    reversescale=True,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '',
    colorbar_title = 'Ozone<br> in sqcm',
))

fig.update_layout(
    title_text='Ozone Distribution',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.55,
        y=0.1,
        xref='paper',
        yref='paper',
        text='Ozone spread',
        showarrow = False
    )]
)

fig.show()

## Is the quality of Air influenced by population and temperature?

In [ ]:
top_cities_population = resultantDf.sort_values('population',ascending=False)
#Top 10 cities with highest population
population_max=top_cities_population.head(n=10)
#10 least population cities
population_min=top_cities_population.tail(n=10)
population_min_max=pd.concat([population_max,population_min],ignore_index=False)
#PairPlot for correlation between Population,Air Quality & temperature
g= sns.pairplot(population_min_max,vars=['population','Air_Quality_Index','temperature'],hue='city_name')
g.fig.suptitle("Correlation between Population,Air_Quality_index and Temperature", fontsize =20)

g.fig.subplots_adjust(top=0.90)

# handles = g._legend_data.values()
# labels = g._legend_data.keys()
# g.fig.legend(handles=handles, labels=labels, loc='upper left', ncol=3)
# g.fig.subplots_adjust(top=0.92, bottom=0.08)